In [14]:
from dotenv import load_dotenv

load_dotenv()

True

LangChain은 도구(Tools)라는 기능을 제공해 준다.

도구는 LLM이 다양한 기능을 수행할 수 있도록 도와주는 유틸리티이다.

예: 인터넷 검색, 프로그래밍 언어 실행, SQL 데이터베이스 조회 등

Tools: https://python.langchain.com/docs/integrations/tools/

In [13]:
from langchain_core.tools import Tool
from langchain_experimental.tools import PythonREPLTool

도구 관련 필요 라이브러리 불러오기

In [ ]:
pip install langchain_experimental

`langchain_experimental.tools`을 불러올 수 없으면

`pip install langchain_experimental` 명령어를 실행해서 설치한다.

In [12]:
python_tool = PythonREPLTool()

python_str_code = "print(100 + 200)"
python_tool.invoke(python_str_code) # '300\n'

'300\n'

`PythonREPLTool`은 문자열을 파이썬 코드로 변환 후 실행까지 수행해주는 도구이다.

In [41]:
def execute_python_str_code(code):
  print("#Code:")
  print(f"{code}\n")
  
  print("#Execute:")
  return python_tool.invoke(code)

langChain의 `RunnableLambda`로 실행할 사용자 정의 함수 정의

In [24]:
execute_python_str_code("print(100 + 200)")

'300\n'

사용자 정의 함수 동작 확인

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are Raymond Hetting, an expert python programmer.\n"
      "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code."
    ),
    ("human", "{user_input}")
  ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser() | RunnableLambda(execute_python_str_code)
user_input = "로또 번호 생성기를 출력하는 코드를 작성하세요."
answer = chain.invoke(user_input)

print(answer)

#Code:
import random

def generate_lotto_numbers():
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    lotto_numbers = generate_lotto_numbers()
    print("로또 번호:", lotto_numbers)

#Execute:



1. `StrOutputParser()`에서 LLM이 생성한 Python 코드를 문자열로 반환한다.

2. 반환 받은 문자열 형태의 파이썬 코드를 `RunnableLambda`를 이용해서 사용자 정의 함수에 값을 넘겨준다.

3. 사용자 정의 함수에서 `PythonREPLTool` 도구로 실행한다.